### Desarrollo de bases de datos

#### BD Tabular

In [1]:
import pandas as pd

In [2]:
data_prod = pd.read_csv('../fuentes_de_informacion/productos.csv')
data_prod.head()

,id_producto,nombre,categoria,subcategoria,marca,precio_usd,stock,color,potencia_w,capacidad,voltaje,peso_kg,garantia_meses,descripcion
0,P0001,Licuadora,Cocina,Preparación,TechHome,283.63,108,Blanco,650.0,1.2L,12V,5.6,36,"Descubrí el poder de la Licuadora de TechHome,..."
1,P0002,Licuadora,Cocina,Preparación,TechHome,1273.06,114,Rosa,300.0,2.0L,220V,35.9,36,"Descubrí el poder de la Licuadora de TechHome,..."
2,P0003,Plus Licuadora Pro,Cocina,Preparación,TechHome,329.07,97,Negro,700.0,1.2L,220V,47.9,18,Descubrí el poder de la Plus Licuadora Pro de ...
3,P0004,Compacto Licuadora,Cocina,Preparación,ChefMaster,259.42,75,Rosa,1000.0,2.0L,220V,48.7,24,Descubrí el poder de la Compacto Licuadora de ...
4,P0005,Licuadora,Cocina,Preparación,HomeChef,2602.78,97,Azul,1350.0,1.2L,110-220V,49.3,12,"Descubrí el poder de la Licuadora de HomeChef,..."


In [3]:
data_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_producto     300 non-null    object 
 1   nombre          300 non-null    object 
 2   categoria       300 non-null    object 
 3   subcategoria    300 non-null    object 
 4   marca           300 non-null    object 
 5   precio_usd      300 non-null    float64
 6   stock           300 non-null    int64  
 7   color           300 non-null    object 
 8   potencia_w      298 non-null    float64
 9   capacidad       58 non-null     object 
 10  voltaje         300 non-null    object 
 11  peso_kg         300 non-null    float64
 12  garantia_meses  300 non-null    int64  
 13  descripcion     300 non-null    object 
dtypes: float64(3), int64(2), object(9)
memory usage: 32.9+ KB


**Datos relevantes:**

Variables categóricas:

In [4]:
len(data_prod['id_producto'].unique())

300

In [5]:
data_prod['categoria'].unique()

array(['Cocina', 'Climatización', 'Lavado', 'Audio y Video'], dtype=object)

In [6]:
data_prod['subcategoria'].unique()

array(['Preparación', 'Cocción', 'Refrigeración',
       'Pequeños Electrodomésticos', 'Aires Acondicionados',
       'Calefacción', 'Ventilación', 'Purificación', 'Lavado de Ropa',
       'Secado', 'Lavado de Vajilla', 'Planchado', 'Televisores'],
      dtype=object)

In [7]:
data_prod['marca'].unique()

array(['TechHome', 'ChefMaster', 'HomeChef', 'KitchenPro', 'CookElite',
       'PureAir', 'EcoClima', 'ClimaTech', 'ThermoControl', 'AirFlow',
       'WashPro', 'SparkleHome', 'CleanMaster', 'LaundryTech',
       'FreshWash', 'VisionPro', 'ScreenPro'], dtype=object)

In [8]:
data_prod['color'].unique()

array(['Blanco', 'Rosa', 'Negro', 'Azul', 'Dorado', 'Gris', 'Plateado',
       'Verde', 'Rojo', 'Amarillo'], dtype=object)

In [9]:
len(data_prod['nombre'].unique())

247

Variables numéricas:

In [10]:
data_prod[['precio_usd', 'stock', 'garantia_meses']].describe()

,precio_usd,stock,garantia_meses
count,300.000000,300.000000,300.000000
mean,1481.669567,99.950000,19.720000
std,845.525639,58.354534,10.112491
min,28.220000,1.000000,6.000000
25%,836.260000,48.250000,12.000000
50%,1409.915000,100.000000,18.000000
75%,2142.560000,150.000000,24.000000
max,2992.330000,200.000000,36.000000


Se tendrán en cuenta los cálculos extraidos para que sean considerados por el LLM. Sin embargo, se procede a chequear la columnas 'capacidad' ya que contiene muchos valores nulos. Se congetura que se debe a que no es una característica correspondiente para algunos productos.

In [11]:
cap = data_prod[data_prod['capacidad'].isna()]
cap['subcategoria'].unique()

array(['Preparación', 'Cocción', 'Refrigeración',
       'Pequeños Electrodomésticos', 'Aires Acondicionados',
       'Calefacción', 'Ventilación', 'Purificación', 'Lavado de Ropa',
       'Secado', 'Planchado', 'Televisores'], dtype=object)

Construcción de base de datos con SQLite

In [12]:
import sqlite3

conn = sqlite3.connect("../databases/productos.db")
cursor = conn.cursor()

In [13]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS productos(
id_producto TEXT PRIMARY KEY,
nombre TEXT,
categoria TEXT,
subcategoria TEXT,
marca TEXT,
precio_usd REAL,
stock INTEGER,
color TEXT,
potencia_w REAL,
capacidad REAL,
voltaje TEXT,
peso_kg REAL,
garantia_meses INTEGER,
descripcion TEXT)""")

conn.commit()

In [14]:
cursor.execute("DELETE FROM productos")
conn.commit()

In [15]:
data_prod.to_sql("productos", conn, if_exists="append", index=False)

300

#### BD Vectorial

In [15]:
#!pip install chromadb sentence-transformers

In [16]:
import json

In [17]:
with open('../fuentes_de_informacion/faqs.json', 'r', encoding="utf-8") as f:
    faqs = json.load(f)
df_faqs = pd.DataFrame(faqs)
df_faqs.head()

,id_faq,id_producto,nombre_producto,categoria,pregunta,respuesta,fecha_publicacion,vistas,util
0,FAQ00001,P0001,Licuadora,Especificaciones,¿Qué voltaje requiere?,El Licuadora funciona con 12V. El consumo es d...,2025-01-05,4067,22
1,FAQ00002,P0001,Licuadora,Mantenimiento,¿Cada cuánto debo hacer mantenimiento?,El Licuadora de TechHome está diseñado para us...,2025-05-18,2556,97
2,FAQ00003,P0001,Licuadora,Uso,¿Es seguro para uso continuo?,El Licuadora de TechHome está diseñado para us...,2024-12-04,1654,19
3,FAQ00004,P0001,Licuadora,Uso,¿Cómo se usa correctamente este producto?,El Licuadora de TechHome está diseñado para us...,2025-08-02,4956,20
4,FAQ00005,P0001,Licuadora,Especificaciones,¿Cuál es el consumo eléctrico?,El Licuadora funciona con 12V. El consumo es d...,2025-06-15,1683,18


In [18]:
df_faqs['contenido'] = (
    "Pregunta: "+df_faqs['pregunta'] +
    "\n Respuesta: "+df_faqs['respuesta']
)

In [19]:
df_faqs["contenido"].apply(lambda x: len(x.split())).describe()

count    3000.000000
mean       36.735667
std         4.251331
min        20.000000
25%        36.000000
50%        38.000000
75%        39.000000
max        43.000000
Name: contenido, dtype: float64

Se decidió unir las preguntas y respuestas y su longitud promedio es 36.7 palabras, observándose baja variabilidad y tamaños muy reducidos. Dado que ningún documento supera límites típicos de chunking para embeddings, se decidió no fragmentar el contenido para preservar coherencia semántica y evitar sobresegmentación innecesaria.

In [20]:
metadatos = df_faqs.apply(
    lambda row: {
        'id_producto': row['id_producto'],
        'nombre_producto': row['nombre_producto'],
        "categoria": row['categoria'],
        'fecha_publicacion': row['fecha_publicacion']},
        axis=1).tolist()

In [21]:
import chromadb
from chromadb.utils import embedding_functions

In [22]:
funcion_embedding = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [32]:
import os

BASE_DIR = os.path.abspath("..")
VECTOR_DB_PATH = os.path.join(BASE_DIR, "databases", "vectores")
os.makedirs(VECTOR_DB_PATH, exist_ok=True)

client = chromadb.PersistentClient(path=VECTOR_DB_PATH)

In [33]:
coleccion = client.get_or_create_collection(
    name='coleccion_faqs',
    embedding_function=funcion_embedding)

In [35]:
coleccion.add(
    documents=df_faqs['contenido'].tolist(),
    metadatas=metadatos,
    ids=df_faqs["id_faq"].astype(str).tolist()
)

In [36]:
coleccion.count()

3000

#### BD de Grafos

In [81]:
BASE_DIR = os.path.abspath("..")
REVIEWS_PATH = os.path.join(BASE_DIR, "fuentes_de_informacion", "resenas_usuarios")

In [82]:
def parsear_resena(ruta_archivo):
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        lineas = f.readlines()
    
    fecha = lineas[0].split(": ", 1)[1].strip()
    usuario = lineas[1].split(": ", 1)[1].strip()
    telefono = lineas[2].split(": ", 1)[1].strip()
    producto = lineas[3].split(": ", 1)[1].strip()
    puntaje = lineas[4].split(": ", 1)[1].strip()
    provincia = lineas[5].split(": ", 1)[1].strip()
    
    texto = "".join(lineas[7:]).strip()
    
    return {
        "fecha": fecha,
        "usuario": usuario,
        "telefono": telefono,
        "producto": producto,
        "puntaje": puntaje,
        "provincia": provincia,
        "texto_resena": texto}

In [83]:
registros = []

for archivo in os.listdir(REVIEWS_PATH):
    if archivo.endswith(".txt"):
        ruta = os.path.join(REVIEWS_PATH, archivo)
        datos = parsear_resena(ruta)
        registros.append(datos)

df_resenas = pd.DataFrame(registros)

In [84]:
df_resenas.head()

,fecha,usuario,telefono,producto,puntaje,provincia,texto_resena
0,2024-04-17,Lucía_Acosta,+54 9 83 9938-6469,Exprimidor 2024 (P0129),5/5,Tierra del Fuego,Hola a todos! Increíble relación calidad-preci...
1,2025-10-03,Nicolás_Vega,+54 9 127 4245-7317,Profesional Batidora de Mano (P0017),5/5,Santa Cruz,Buenas! Mejor de lo que esperaba con Profesion...
2,2025-10-11,Lucía_Flores,+54 9 118 1905-8647,Plancha Seca 2024 (P0285),4/5,Santiago del Estero,"Quiero compartir mi experiencia. Me encantó, y..."
3,2025-05-25,Miranda_Fernández,+54 9 326 9566-1668,Advanced Procesadora II (P0012),5/5,Río Negro,"Hola a todos! Funciona de 10, muy satisfecho/a..."
4,2025-06-03,Rodrigo_Molina,+54 9 225 6511-1859,Elite Plancha Seca X (P0284),5/5,CABA,Buenas! No me arrepiento para nada de la compr...


Se continúa con el preprocesamiento de este dataframe antes de construir la base de datos de grafos.

In [85]:
df_resenas['fecha'] = pd.to_datetime(df_resenas['fecha'])

In [86]:
df_resenas['usuario'] = df_resenas['usuario'].str.replace('_', ' ', regex=False)

In [87]:
df_resenas[['nombre_producto', 'id_producto']] = df_resenas['producto'].str.extract(r'^(.*?)\s\((P\d+)\)$')

In [88]:
df_resenas['puntaje'] = df_resenas['puntaje'].str.split('/').str[0].astype(int)

In [89]:
df_resenas = df_resenas.drop(columns=['telefono', 'producto'])

In [90]:
df_resenas.head()

,fecha,usuario,puntaje,provincia,texto_resena,nombre_producto,id_producto
0,2024-04-17,Lucía Acosta,5,Tierra del Fuego,Hola a todos! Increíble relación calidad-preci...,Exprimidor 2024,P0129
1,2025-10-03,Nicolás Vega,5,Santa Cruz,Buenas! Mejor de lo que esperaba con Profesion...,Profesional Batidora de Mano,P0017
2,2025-10-11,Lucía Flores,4,Santiago del Estero,"Quiero compartir mi experiencia. Me encantó, y...",Plancha Seca 2024,P0285
3,2025-05-25,Miranda Fernández,5,Río Negro,"Hola a todos! Funciona de 10, muy satisfecho/a...",Advanced Procesadora II,P0012
4,2025-06-03,Rodrigo Molina,5,CABA,Buenas! No me arrepiento para nada de la compr...,Elite Plancha Seca X,P0284


Construcción de base de datos con Neo4j

In [38]:
#!pip install neo4j

In [91]:
from neo4j import GraphDatabase

In [101]:
uri = "neo4j+s://a5a91658.databases.neo4j.io"
user = "a5a91658"
password = "pD3DBz-E3P3MRLaKf077DwxQFhi3KEIl1EfE2hxF2LY"

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    print(session.run("RETURN 1").single()[0])

1


In [41]:
with driver.session() as session:
    session.run("""
    CREATE CONSTRAINT usuario_id IF NOT EXISTS
    FOR (u:Usuario) REQUIRE u.id_usuario IS UNIQUE
    """)

    session.run("""
    CREATE CONSTRAINT producto_id IF NOT EXISTS
    FOR (p:Producto) REQUIRE p.id_producto IS UNIQUE
    """)

In [42]:
data = df_resenas.to_dict("records")

In [43]:
def cargar_resenas(tx, rows):
    query = """
    UNWIND $rows AS row

    MERGE (u:Usuario {id_usuario: row.usuario})

    MERGE (p:Producto {id_producto: row.id_producto})
    SET p.nombre = row.nombre_producto

    CREATE (r:Reseña {
        fecha: date(row.fecha),
        puntaje: toInteger(row.puntaje),
        texto: row.texto_resena,
        provincia: row.provincia
    })

    MERGE (u)-[:ESCRIBIO]->(r)
    MERGE (r)-[:SOBRE]->(p)
    """
    tx.run(query, rows=rows)

with driver.session() as session:
    session.execute_write(cargar_resenas, data)

### Interfaces de consulta

#### Función de consulta - BD Vectorial

In [55]:
def buscar_faqs(consulta, filtros=None, k=3):
    resultados = coleccion.query(
        query_texts=[consulta],
        n_results=k,
        where=filtros)

    return {"documentos": resultados["documents"][0],
        "metadatos": resultados["metadatas"][0],
        "distancias": resultados["distances"][0]}

In [57]:
#Prueba:
buscar_faqs(consulta="¿Es normal que se caliente?",
    filtros={"nombre_producto": "Licuadora"},
    k=3)

{'documentos': ['Pregunta: ¿Cuál es el consumo eléctrico?\nRespuesta: El Licuadora funciona con 12V. El consumo es de 650W. Recomendamos usar un estabilizador de tensión.',
  'Pregunta: ¿Cuál es el consumo eléctrico?\nRespuesta: El Licuadora funciona con 110-220V. El consumo es de 1350W. Recomendamos usar un estabilizador de tensión.',
  'Pregunta: ¿Qué voltaje requiere?\nRespuesta: El Licuadora funciona con 12V. El consumo es de 650W. Recomendamos usar un estabilizador de tensión.'],
 'metadatos': [{'categoria': 'Especificaciones',
   'nombre_producto': 'Licuadora',
   'fecha_publicacion': '2025-06-15',
   'id_producto': 'P0001'},
  {'id_producto': 'P0005',
   'fecha_publicacion': '2025-08-15',
   'nombre_producto': 'Licuadora',
   'categoria': 'Especificaciones'},
  {'categoria': 'Especificaciones',
   'fecha_publicacion': '2025-01-05',
   'nombre_producto': 'Licuadora',
   'id_producto': 'P0001'}],
 'distancias': [0.7439897060394287, 0.7515973448753357, 0.7902710437774658]}

#### Función de consulta - BD Tabular

In [52]:
system_prompt = """
Sos un experto en bases de datos SQL.  
A partir de una consulta en lenguaje natural, debes generar solamente un objeto JSON con filtros válidos para la tabla productos, sin más comentarios explicativos. 
Si no se identifican filtros válidos, debes devolver {}.

Reglas:

Columnas existentes: 'id_producto', 'nombre', 'categoria', 'subcategoria', 'marca', 'precio_usd', 'stock', 'color', 'potencia_w', 'capacidad', 'voltaje', 'peso_kg', 'garantia_meses', 'descripcion'.
Las categorías válidas son: Cocina, Climatización, Lavado, Audio y Video.
Las subcategorías válidas son: 'Preparación', 'Cocción', 'Refrigeración','Pequeños Electrodomésticos', 'Aires Acondicionados','Calefacción', 'Ventilación', 'Purificación', 'Lavado de Ropa','Secado', 'Lavado de Vajilla', 'Planchado', 'Televisores'
Tiene 300 registros.

Estructura obligatoria del JSON:
- Igualdad:
  "columna": "valor"

- Comparación numérica (solo válido para precio_usd, stock, potencia_w, capacidad, voltaje, peso_kg y garantia_meses):
  "columna": {"<": valor}
  "columna": {"<=": valor}
  "columna": {">": valor}
  "columna": {">=": valor}

Ejemplo correcto:
Consulta:
"Productos de cocina con precio menor a 500 dólares"

Respuesta:
{
  "categoria": "Cocina",
  "precio_usd": {"<": 500}
}

Devuelve únicamente JSON válido, sin textos explicativos o comentarios, antes o después del objeto JSON.
Si no hay filtros, devuelve {}.
"""

In [38]:
import requests
import re

El modelo Llama 3.2 genera un filtro en base al texto del usuario, en un formato JSON.

In [39]:
def generar_filtros(consulta_usuario):
    url = "http://localhost:11434/api/generate"
    
    prompt_completo = f"{system_prompt}\n\nConsulta del usuario:\n{consulta_usuario}"
    
    payload = {
        "model": "llama3.2",
        "prompt": prompt_completo,
        "stream": False,
        "temperature": 0
    }
    
    response = requests.post(url, json=payload)
    resultado = response.json()["response"]
    
    return resultado

In [69]:
#Prueba
consulta = "Mostrar productos de climatización con precio menor a 1000 dólares y garantía mayor a 12 meses"
res1 = generar_filtros(consulta)
res1

'{\n  "categoria": "Climatización",\n  "precio_usd": {"<": 1000},\n  "garantia_meses": {">": 12}\n}'

In [65]:
#Prueba2
consulta2 = "Mostrar lavarropas de menos de 200 dólares, con al menos 6 meses de garantía"
res2 = generar_filtros(consulta2)
res2

'{\n  "categoria": "Lavado",\n  "precio_usd": {"<": 200},\n  "garantia_meses": {">": 6}\n}'

A través de una función intermedia, se asegura el formato del filtro pedido.

In [70]:
filtros_json = json.loads(res2)

Se construyen los filtros para usar como WHERE sobre la base de datos tabular.

In [72]:
def construir_where(filtros):
    condiciones = []
    valores = []

    for columna, valor in filtros.items():

        # Caso igualdad
        if not isinstance(valor, dict):
            condiciones.append(f"{columna} = ?")
            valores.append(valor)

        # Caso comparación
        else:
            for operador, numero in valor.items():
                condiciones.append(f"{columna} {operador} ?")
                valores.append(numero)

    if not condiciones:
        return "", []

    where_clause = "WHERE " + " AND ".join(condiciones)
    return where_clause, valores

In [73]:
where, params = construir_where(filtros_json)

query = f"SELECT * FROM productos {where};"

cursor.execute(query, params)
resultados = cursor.fetchall()
resultados

[('P0229',
  'Deluxe Lavarropas Automático 2024',
  'Lavado',
  'Lavado de Ropa',
  'WashPro',
  191.93,
  2,
  'Plateado',
  300.0,
  '8kg',
  '110-220V',
  19.2,
  24,
  'Presentamos la Deluxe Lavarropas Automático 2024 de WashPro, un producto excepcional de la categoría Lavado. Con 300W de potencia y tecnología de vanguardia, este electrodoméstico está diseñado para facilitar tu vida diaria. Construcción robusta y materiales de primera calidad garantizan años de uso confiable. Interfaz intuitiva y controles ergonómicos para una experiencia de usuario superior. Sistema de seguridad certificado y múltiples protecciones contra sobrecalentamiento. Eficiencia energética clase A+ para reducir el consumo eléctrico. Diseño moderno que se integra perfectamente en cualquier ambiente. Garantía de 24 meses respaldada por WashPro. '),
 ('P0233',
  'Lavarropas Automático',
  'Lavado',
  'Lavado de Ropa',
  'LaundryTech',
  184.74,
  111,
  'Rojo',
  550.0,
  '10kg',
  '220V',
  25.3,
  18,
  'Pre

Interfaz de consulta final:

In [74]:
def buscar_productos(consulta_usuario, x=conn):

    # Filtros con Llama
    respuesta = generar_filtros(consulta_usuario)

    # Respuesta a JSON
    try:
        filtros = json.loads(respuesta)
    except:
        return "Error: el modelo no devolvió JSON válido."

    # Validación básica de columnas permitidas
    columnas_validas = {
        'id_producto', 'nombre', 'categoria', 'subcategoria',
        'marca', 'precio_usd', 'stock', 'color',
        'potencia_w', 'capacidad', 'voltaje',
        'peso_kg', 'garantia_meses', 'descripcion'}

    for columna in filtros.keys():
        if columna not in columnas_validas:
            return f"Error: columna inválida detectada: {columna}"

    # Construir cláusula WHERE
    where_clause, valores = construir_where(filtros)

    # Query sobre BD Tab
    query = f"SELECT * FROM productos {where_clause};"

    cursor = x.cursor()
    cursor.execute(query, valores)
    resultados = cursor.fetchall()

    return resultados

In [80]:
consulta = "Mostrar productos de cocción con precio mayor a 100 dólares"
buscar_productos(consulta)

[('P0013',
  'Procesadora',
  'Cocina',
  'Preparación',
  'KitchenPro',
  82.93,
  92,
  'Azul',
  1700.0,
  '2.0L',
  '220V',
  30.1,
  24,
  'La Procesadora de KitchenPro es tu aliada perfecta para cualquier preparación culinaria. Equipada con un motor profesional de 1700W, procesa desde vegetales delicados hasta masas pesadas. El bowl de 2.0L en policarbonato libre de BPA asegura preparaciones seguras para toda la familia. Incluye 15 accesorios intercambiables: disco rallador fino, disco rallador grueso, disco rebanador ajustable, cuchilla multipropósito en S, batidor de claras, gancho amasador, exprimidor de cítricos, y más. Sistema de seguridad con triple bloqueo evita el encendido accidental. Panel de control digital con display LED muestra tiempo, velocidad y programa seleccionado. 12 programas preestablecidos para pan, pizza, pasteles, merengues, mayonesas y más. La tecnología SilentTech reduce el ruido hasta en un 60% comparado con modelos tradicionales. Cable retráctil de 1.

#### Función de consulta - BD de Grafos

In [164]:
system_prompt_grafos = """
Sos un generador de consultas Cypher para Neo4j.

Estructura del grafo:

Nodos:
(:Usuario {id_usuario, nombre, provincia})
(:Reseña {puntaje, texto, fecha})
(:Producto {id_producto, nombre})

Relaciones:
(:Usuario)-[:ESCRIBIO]->(:Reseña)
(:Reseña)-[:SOBRE]->(:Producto)

Camino válido en el grafo:
(:Usuario)-[:ESCRIBIO]->(:Reseña)-[:SOBRE]->(:Producto)

No existe relación directa entre Usuario y Producto.
Toda consulta que vincule usuarios y productos debe pasar obligatoriamente por el nodo Reseña.

Reglas estrictas:
- Devuelve únicamente una consulta Cypher válida, sin explicaciones, ni al comienzo ni al final.
- Usa únicamente los labels, relaciones y propiedades exactamente como están escritos arriba, sin inventar nodos, relaciones ni propiedades.
- Respeta mayúsculas, minúsculas y tildes.
- Siempre utiliza MATCH explícito.
- Devuelve resultados con RETURN explícito.
- La consulta debe ser exclusivamente de lectura, sin CREATE, DELETE, MERGE, SET, CALL ni procedimientos externos.

Relaciones dirigidas:
- ESCRIBIO va desde (:Usuario) hacia (:Reseña).
- SOBRE va desde (:Reseña) hacia (:Producto).
- La dirección de las flechas es obligatoria y no debe invertirse.
- Solo usa -[:REL]- si necesitas ignorar dirección explícitamente.

Ejemplos:

Consulta:
"Usuarios que escribieron reseñas con puntaje mayor a 4"
Respuesta:
MATCH (u:Usuario)-[:ESCRIBIO]->(r:Reseña)
WHERE r.puntaje > 4
RETURN u.nombre, r.puntaje

Consulta:
"Productos con reseñas con puntaje igual a 2"
Respuesta:
MATCH (r:Reseña)-[:SOBRE]->(p:Producto)
WHERE r.puntaje = 2
RETURN p.nombre

Fin de ejemplos.
Ahora genera la consulta para la siguiente entrada.
"""

Se asegura que la respuesta de la LLM tenga el formato limpio, sin agregados.

In [165]:
def limpiar_cypher(respuesta_modelo):
    # Busca bloque ```cypher ... ``` o ``` ... ```
    match = re.search(r"```(?:cypher)?\s*(.*?)```", respuesta_modelo, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Si no hay markdown, devuelve el texto limpio
    return respuesta_modelo.strip()

In [166]:
def generar_cypher(consulta_usuario):
    prompt = f"{system_prompt_grafos}\n\nConsulta:\n{consulta_usuario}"

    payload = {
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False,
        "temperature": 0}

    response = requests.post("http://localhost:11434/api/generate", json=payload)
    respuesta_cruda = response.json()["response"]
    
    return limpiar_cypher(respuesta_cruda)

Prueba de la respuesta del LLM con respecto a las query Cypher.

In [167]:
consulta_gr = "Mostrar productos cuyas reseñas son iguales a 2"
print(generar_cypher(consulta_gr))

MATCH (r:Reseña)-[:SOBRE]->(p:Producto)
WHERE r.puntaje = 2
RETURN p.nombre


In [168]:
def ejecutar_cypher(query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

Interfaz final para el usuario.

In [169]:
def buscar_en_grafo(consulta_usuario):
    cypher = generar_cypher(consulta_usuario)
    resultados = ejecutar_cypher(cypher)
    return resultados

In [170]:
print(buscar_en_grafo(consulta_gr))

[{'p.nombre': 'Digital Sandwichera Plus'}, {'p.nombre': 'Ventilador de Torre Pro'}, {'p.nombre': 'Olla de Cocción Lenta'}, {'p.nombre': 'Ionizador 3000'}, {'p.nombre': 'Max Conservadora'}, {'p.nombre': 'Pro Deshumidificador Pro'}, {'p.nombre': 'Advanced Freidora de Aire'}, {'p.nombre': 'Deluxe Deshumidificador'}, {'p.nombre': 'Premium Heladera'}, {'p.nombre': 'Centrifuga'}, {'p.nombre': 'Advanced Humidificador X'}, {'p.nombre': 'Mixer II'}, {'p.nombre': 'Deluxe Centrifuga'}, {'p.nombre': 'Licuadora'}, {'p.nombre': 'Aire Split'}, {'p.nombre': 'Frigobar'}, {'p.nombre': 'Plus Lavarropa Carga Frontal II'}, {'p.nombre': 'Smart Conservadora 2024'}, {'p.nombre': 'Ventilador de Torre 3000'}, {'p.nombre': 'Plancha a Vapor 3000'}, {'p.nombre': 'Ventilador de Techo Pro'}, {'p.nombre': 'Lavavajillas'}, {'p.nombre': 'Pava Eléctrica'}, {'p.nombre': 'Lavavajillas Compacto'}, {'p.nombre': 'Abridor de Latas'}, {'p.nombre': 'Pro Olla Arrocera'}, {'p.nombre': 'Plus Parrilla Eléctrica'}, {'p.nombre': 'Tur